In [1]:
# Import Pandas and NumPy
import pandas as pd
import numpy as np

# Import SQL Alchemy
from sqlalchemy import create_engine
import psycopg2

# Import Password
from config import password

In [2]:
conn = (f"postgres:{password}@localhost:5432/restaurant_rating_db")
engine = create_engine(f'postgresql://{conn}')
conn = engine.connect()

In [3]:
csv_file = "Resources/geoplaces2.csv"
places_df = pd.read_csv(csv_file, encoding="utf-8")
places_df.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rinc�n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


Data Cleaning

In [4]:
places_df.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')

In [5]:
places_df = places_df[["placeID", "latitude", "longitude", "name", "city", "alcohol", "smoking_area",
                      "accessibility", "Rambience"]]
places_df.head()

,placeID,latitude,longitude,name,city,alcohol,smoking_area,accessibility,Rambience
0,134999,18.915421,-99.184871,Kiku Cuernavaca,Cuernavaca,No_Alcohol_Served,none,no_accessibility,familiar
1,132825,22.147392,-100.983092,puesto de tacos,s.l.p.,No_Alcohol_Served,none,completely,familiar
2,135106,22.149709,-100.976093,El Rinc�n de San Francisco,San Luis Potosi,Wine-Beer,only at bar,partially,familiar
3,132667,23.752697,-99.163359,little pizza Emilio Portes Gil,victoria,No_Alcohol_Served,none,completely,familiar
4,132613,23.752903,-99.165076,carnitas_mata,victoria,No_Alcohol_Served,permitted,completely,familiar


In [6]:
places_df = places_df.rename(columns={"Rambience": "ambience"})
places_df = places_df.rename(columns={"placeID": "placeid"})

In [7]:
places_df[places_df["city"] == "?"]

,placeid,latitude,longitude,name,city,alcohol,smoking_area,accessibility,ambience
7,132875,22.149901,-100.993779,shi ro ie,?,Wine-Beer,section,no_accessibility,familiar
27,132561,23.726819,-99.126506,cafe ambar,?,No_Alcohol_Served,none,completely,familiar
31,134975,18.940828,-99.215426,Rincon del Bife,?,Full_Bar,section,completely,familiar
43,132564,23.730925,-99.145185,churchs,?,No_Alcohol_Served,none,completely,familiar
45,132925,22.153500,-100.976243,el pueblito,?,Wine-Beer,not permitted,completely,familiar
62,132723,22.148934,-101.019845,Gordas de morales,?,Full_Bar,section,completely,familiar
70,132884,22.139578,-101.027886,dairy queen,?,No_Alcohol_Served,none,completely,familiar
72,135104,23.752982,-99.168434,vips,?,Full_Bar,not permitted,completely,familiar
83,132717,23.731860,-99.150436,tortas hawai,?,No_Alcohol_Served,not permitted,partially,familiar
87,132830,22.150849,-100.939752,Rincon Huasteco,?,No_Alcohol_Served,none,completely,familiar


In [8]:
places_df["smoking_area"].unique()

array(['none', 'only at bar', 'permitted', 'section', 'not permitted'],
      dtype=object)

In [9]:
places_df = places_df.replace("?", "Not Recorded")

In [10]:
# check all columns with any missing/null values
places_df.isna().sum()

placeid          0
latitude         0
longitude        0
name             0
city             0
alcohol          0
smoking_area     0
accessibility    0
ambience         0
dtype: int64

In [11]:
# check all duplicate rows
duplicate_rows_df = places_df[places_df.duplicated()]
print (f"Number of duplicate rows: {duplicate_rows_df.shape}")

Number of duplicate rows: (0, 9)


In [12]:
# City coulm had lot of variations for "San Luis Potosi"
places_df["city"].unique()

array(['Cuernavaca', 's.l.p.', 'San Luis Potosi', 'victoria ', 'victoria',
       'Cd Victoria', 'Not Recorded', 'san luis potosi', 'Jiutepec',
       'cuernavaca', 'slp', 'Soledad', 'san luis potos',
       'san luis potosi ', 'Ciudad Victoria', 'Cd. Victoria', 's.l.p'],
      dtype=object)

In [13]:
# Replace the values 's.l.p.', 'san luis potosi', 'slp', 'san luis potos', 'san luis potosi ' and 's.l.p' 
# with "San Luis Potosi" to get the correct data loaded into Postgres for further analysis
places_df = places_df.replace(['s.l.p.','san luis potosi','slp','san luis potos','san luis potosi ','s.l.p'], "San Luis Potosi")
places_df["city"].unique()

array(['Cuernavaca', 'San Luis Potosi', 'victoria ', 'victoria',
       'Cd Victoria', 'Not Recorded', 'Jiutepec', 'cuernavaca', 'Soledad',
       'Ciudad Victoria', 'Cd. Victoria'], dtype=object)

In [14]:
places_df = places_df.replace('cuernavaca', "Cuernavaca")
places_df = places_df.replace(['victoria ','victoria','Cd Victoria','Cd. Victoria'], 'Ciudad Victoria')
places_df["city"].unique()

array(['Cuernavaca', 'San Luis Potosi', 'Ciudad Victoria', 'Not Recorded',
       'Jiutepec', 'Soledad'], dtype=object)

In [15]:
places_df.head(10)

,placeid,latitude,longitude,name,city,alcohol,smoking_area,accessibility,ambience
0,134999,18.915421,-99.184871,Kiku Cuernavaca,Cuernavaca,No_Alcohol_Served,none,no_accessibility,familiar
1,132825,22.147392,-100.983092,puesto de tacos,San Luis Potosi,No_Alcohol_Served,none,completely,familiar
2,135106,22.149709,-100.976093,El Rinc�n de San Francisco,San Luis Potosi,Wine-Beer,only at bar,partially,familiar
3,132667,23.752697,-99.163359,little pizza Emilio Portes Gil,Ciudad Victoria,No_Alcohol_Served,none,completely,familiar
4,132613,23.752903,-99.165076,carnitas_mata,Ciudad Victoria,No_Alcohol_Served,permitted,completely,familiar
5,135040,22.135617,-100.969709,Restaurant los Compadres,San Luis Potosi,Wine-Beer,none,no_accessibility,familiar
6,132732,23.754357,-99.171288,Taqueria EL amigo,Ciudad Victoria,No_Alcohol_Served,none,completely,familiar
7,132875,22.149901,-100.993779,shi ro ie,Not Recorded,Wine-Beer,section,no_accessibility,familiar
8,132609,23.760268,-99.165865,Pollo_Frito_Buenos_Aires,Ciudad Victoria,No_Alcohol_Served,not permitted,completely,quiet
9,135082,22.151448,-100.915099,la Estrella de Dimas,San Luis Potosi,No_Alcohol_Served,none,no_accessibility,familiar


Database

In [16]:
# Insert data into Geoplaces table
places_df.to_sql(name='geoplaces', con=engine, if_exists='append', index=False)

In [17]:
# Query the data in Postgres
geoplaces = pd.read_sql("SELECT * FROM GEOPLACES", conn)
geoplaces.head(10)

,placeid,latitude,longitude,name,city,alcohol,smoking_area,accessibility,ambience
0,134999,18.915421,-99.184871,Kiku Cuernavaca,Cuernavaca,No_Alcohol_Served,none,no_accessibility,familiar
1,132825,22.147392,-100.983092,puesto de tacos,San Luis Potosi,No_Alcohol_Served,none,completely,familiar
2,135106,22.149709,-100.976093,El Rinc�n de San Francisco,San Luis Potosi,Wine-Beer,only at bar,partially,familiar
3,132667,23.752697,-99.163359,little pizza Emilio Portes Gil,Ciudad Victoria,No_Alcohol_Served,none,completely,familiar
4,132613,23.752903,-99.165076,carnitas_mata,Ciudad Victoria,No_Alcohol_Served,permitted,completely,familiar
5,135040,22.135617,-100.969709,Restaurant los Compadres,San Luis Potosi,Wine-Beer,none,no_accessibility,familiar
6,132732,23.754357,-99.171288,Taqueria EL amigo,Ciudad Victoria,No_Alcohol_Served,none,completely,familiar
7,132875,22.149901,-100.993779,shi ro ie,Not Recorded,Wine-Beer,section,no_accessibility,familiar
8,132609,23.760268,-99.165865,Pollo_Frito_Buenos_Aires,Ciudad Victoria,No_Alcohol_Served,not permitted,completely,quiet
9,135082,22.151448,-100.915099,la Estrella de Dimas,San Luis Potosi,No_Alcohol_Served,none,no_accessibility,familiar
